# Deep Q-Learning Applied to Algorithmic Trading

<a href="https://www.kaggle.com/addarm/unsupervised-learning-as-signals-for-pairs-trading" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

INTRO


This deep learning network was inspired by the paper:
```BibTeX
@article{theate2021application,
  title={An application of deep reinforcement learning to algorithmic trading},
  author={Th{\'e}ate, Thibaut and Ernst, Damien},
  journal={Expert Systems with Applications},
  volume={173},
  pages={114632},
  year={2021},
  publisher={Elsevier}
}
```

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

IS_KAGGLE = os.getenv('IS_KAGGLE', 'True') == 'True'
if IS_KAGGLE:
    # Kaggle confgs
    print('Running in Kaggle...')
    %pip install scikit-learn
    %pip install tensorflow
    %pip install tqdm
    %pip install matplotlib
    %pip install python-dotenv
    %pip install yfinance
    %pip install pyarrow
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

    DATA_DIR = "/kaggle/input/DATASET"
else:
    DATA_DIR = "./data/"
    print('Running Local...')

import numpy as np
import yfinance as yf
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import BDay
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import skew, kurtosis
import pyarrow as pa
import pyarrow.parquet as pq

os.getcwd()

Running Local...


'/mnt/c/Users/adamd/workspace/deep-reinforced-learning'

In [2]:
START_DATE = "2017-01-01"
SPLIT_DATE = '2018-1-1' # Turning point from train to tst
END_DATE = "2019-12-31" # pd.Timestamp(datetime.now() - BDay(1)).strftime('%Y-%m-%d')
DATA_DIR = "./data"
INDEX = "Date"
TICKER_SYMBOLS = [
    'DIA',  # Dow Jones
    'SPY',  # S&P 500
    'QQQ',  # NASDAQ 100
    'EZU',  # FTSE 100
    'EWJ',  # Nikkei 225
    'GOOGL',  # Google
    'AAPL',  # Apple
    'META',  # Facebook
    'AMZN',  # Amazon
    'MSFT',  # Microsoft
    'NOK',  # Nokia
    'PHIA.AS',  # Philips
    'SIE.DE',  # Siemens
    'BIDU',  # Baidu
    'BABA',  # Alibaba
    '0700.HK',  # Tencent
    '6758.T',  # Sony
    'JPM',  # JPMorgan Chase
    'HSBC',  # HSBC
    '0939.HK',  # CCB
    'XOM',  # ExxonMobil
    'TSLA',  # Tesla
    'VOW3.DE',  # Volkswagen
    '7203.T',  # Toyota
    'KO',  # Coca Cola
    'ABI.BR',  # AB InBev
    '2503.T',  # Kirin
]
TICKER_SYMBOLS = ['TSLA']
TARGET = 'TSLA'
INTERVAL = "1d"

CAPITAL = 100000
STATE_LEN = 30
FEES = 0.1 / 100
FEATURES = 4 # 4 dims: HLOC
OBS_SPACE = (STATE_LEN)*FEATURES
ACT_SPACE = 2
EPISODES = 50

# Financial Data

In [3]:
def get_tickerdata(tickers_symbols, start=START_DATE, end=END_DATE, interval=INTERVAL, datadir=DATA_DIR):
    tickers = {}
    earliest_end= datetime.strptime(end,'%Y-%m-%d')
    latest_start = datetime.strptime(start,'%Y-%m-%d')
    os.makedirs(DATA_DIR, exist_ok=True)
    for symbol in tickers_symbols:
        cached_file_path = f"{datadir}/{symbol}-{start}-{end}-{interval}.csv"

        try:
            if os.path.exists(cached_file_path):
                df = pd.read_parquet(cached_file_path)
                df.index = pd.to_datetime(df.index)
                assert len(df) > 0
            else:
                df = yf.download(
                    symbol,
                    start=START_DATE,
                    end=END_DATE,
                    progress=False,
                    interval=INTERVAL,
                )
                assert len(df) > 0
                df.to_parquet(cached_file_path, index=False, compression="snappy")
            min_date = df.index.min()
            max_date = df.index.max()
            nan_count = df["Close"].isnull().sum()
            skewness = round(skew(df["Close"].dropna()), 2)
            kurt = round(kurtosis(df["Close"].dropna()), 2)
            outliers_count = (df["Close"] > df["Close"].mean() + (3 * df["Close"].std())).sum()
            print(
                f"{symbol} => min_date: {min_date}, max_date: {max_date}, kurt:{kurt}, skewness:{skewness}, outliers_count:{outliers_count},  nan_count: {nan_count}"
            )
            tickers[symbol] = df

            if min_date > latest_start:
                latest_start = min_date
            if max_date < earliest_end:
                earliest_end = max_date
        except Exception as e:
            print(f"Error with {symbol}: {e}")

    return tickers, latest_start, earliest_end

tickers, latest_start, earliest_end = get_tickerdata(TICKER_SYMBOLS)
tickers[TARGET]

TSLA => min_date: 1970-01-01 00:00:00, max_date: 1970-01-01 00:00:00.000000752, kurt:-0.56, skewness:-0.28, outliers_count:0,  nan_count: 0


,Open,High,Low,Close,Adj Close,Volume
1970-01-01 00:00:00.000000000,14.324000,14.688667,14.064000,14.466000,14.466000,88849500
1970-01-01 00:00:00.000000001,14.316667,15.200000,14.287333,15.132667,15.132667,168202500
1970-01-01 00:00:00.000000002,15.094667,15.165333,14.796667,15.116667,15.116667,88675500
1970-01-01 00:00:00.000000003,15.128667,15.354000,15.030000,15.267333,15.267333,82918500
1970-01-01 00:00:00.000000004,15.264667,15.461333,15.200000,15.418667,15.418667,59692500
...,...,...,...,...,...,...
1970-01-01 00:00:00.000000748,27.452000,28.134001,27.333332,27.948000,27.948000,199794000
1970-01-01 00:00:00.000000749,27.890667,28.364668,27.512667,28.350000,28.350000,120820500
1970-01-01 00:00:00.000000750,28.527332,28.898666,28.423332,28.729334,28.729334,159508500
1970-01-01 00:00:00.000000751,29.000000,29.020666,28.407333,28.691999,28.691999,149185500


# Trading Environment

In [4]:
from tf_agents.environments import py_environment, utils
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
import numpy as np

ACT_LONG = 1
ACT_SHORT = -1
ACT_HOLD = 0


class TradingEnv(py_environment.PyEnvironment):
    """
    A custom trading environment for reinforcement learning, compatible with tf_agents.

    This environment simulates a simple trading scenario where an agent can take one of three actions:
    - Long (buy), Short (sell), or Hold a financial instrument, aiming to maximize profit through trading decisions.

    Parameters:
    - data: DataFrame containing the stock market data.
    - data_dim: Dimension of the data to be used for each observation.
    - startingDate: The starting date of the trading period.
    - money: Initial capital to start trading.
    - stateLength: Number of past observations to consider for the state.
    - transactionCosts: Costs associated with trading actions.
    - endingDate: The ending date of the trading period.
    """

    def __init__(self, data, data_dim, startingDate, money, stateLength, transactionCosts, endingDate):
        super(TradingEnv, self).__init__()
        self.data = data
        self.data_dim = data_dim
        self.startingDate = startingDate
        self.initial_balance = money
        self.state_length = stateLength
        self.transaction_cost = transactionCosts
        self.endingDate = endingDate
        self._episode_ended = False

        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=ACT_SHORT, maximum=ACT_LONG, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(shape=(self.state_length, self.data_dim), dtype=np.float32, name='observation')

        self.reset()

    def action_spec(self):
        """Provides the specification of the action space."""
        return self._action_spec

    def observation_spec(self):
        """Provides the specification of the observation space."""
        return self._observation_spec

    def _reset(self):
        """Resets the environment state and prepares for a new episode."""
        self.balance = self.initial_balance
        self.position = 0
        self.total_shares = 0
        self.current_step = self.state_length
        self._episode_ended = False
        initial_observation = self._next_observation()
        return ts.restart(initial_observation)

    def _next_observation(self):
        """Generates the next observation based on the current step."""
        if self.current_step + self.state_length > len(self.data):
            padding_rows_needed = max(len(self.data) - (self.current_step + self.state_length), 0)
            frame = self.data.iloc[-self.state_length:] if padding_rows_needed == 0 else self.data.iloc[-padding_rows_needed:]
            padding = np.zeros((padding_rows_needed, self.data_dim))
            obs = np.vstack((padding, frame[['Close', 'Low', 'High', 'Volume']].values))
        else:
            frame = self.data.iloc[self.current_step-self.state_length:self.current_step]
            obs = frame[['Close', 'Low', 'High', 'Volume']].values

        obs = np.array(obs, dtype=np.float32)

        return obs

    def _step(self, action):
        """Executes a trading action and returns the new state of the environment."""
        if self._episode_ended:
            return self.reset()

        current_price = self.data.iloc[self.current_step]['Close']
        self.current_step += 1

        reward = 0

        if action == ACT_SHORT or action == ACT_LONG:
            if self.total_shares > 0:
                self.balance += self.total_shares * current_price * (1 - self.transaction_cost)
                reward = self.balance - self.initial_balance
                self.total_shares = 0

        if action == ACT_LONG:
            self.position = 1
            self.total_shares = self.balance // (current_price * (1 + self.transaction_cost))
            self.balance -= self.total_shares * current_price * (1 + self.transaction_cost)
        elif action == ACT_SHORT:
            self.position = -1

        done = self.current_step >= len(self.data)
        if done:
            self._episode_ended = True
            return ts.termination(self._next_observation(), reward)
        else:
            return ts.transition(self._next_observation(), reward, discount=1.0)

    def render(self, mode='human'):
        """Outputs the current state of the environment for visualization."""
        print(f'Step: {self.current_step}, Balance: {self.balance}')

environment = TradingEnv(tickers[TARGET], FEATURES, START_DATE, CAPITAL, STATE_LEN, FEES, END_DATE)
utils.validate_py_environment(environment, episodes=5)

2024-03-08 03:25:39.901814: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 03:25:39.934400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 03:25:39.934429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 03:25:39.935285: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 03:25:39.940507: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 03:25:39.941165: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment

tf_env = tf_py_environment.TFPyEnvironment(environment)

print(isinstance(tf_env, tf_environment.TFEnvironment))
print("TimeStep Specs:", tf_env.time_step_spec())
print("Action Specs:", tf_env.action_spec())

True
TimeStep Specs: TimeStep(
{'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(30, 4), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32))})
Action Specs: BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(-1, dtype=int32), maximum=array(1, dtype=int32))


# Deep Q-Network Architecure

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from collections import deque
import random
import tensorflow as tf
tf.get_logger().setLevel('INFO')

# Replay Memory
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

# DQN Model
class DQNModel(Sequential):
    def __init__(self, flattened_observation_space, action_space, learning_rate=0.0001):
        super(DQNModel, self).__init__()
        self.add(Dense(512, activation='relu', input_shape=(flattened_observation_space,)))
        self.add(BatchNormalization())
        self.add(Dropout(0.2))
        self.add(Dense(512, activation='relu'))
        self.add(BatchNormalization())
        self.add(Dropout(0.2))
        self.add(Dense(action_space, activation='linear'))
        self.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))

# DQN Agent
class DQNAgent:
    def __init__(self, observation_space, action_space, replay_memory_size=10000, batch_size=64, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.9995, learning_rate=0.001, target_update_iter=100):
        self.action_space = action_space
        self.memory = ReplayMemory(replay_memory_size)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update_iter = target_update_iter
        self.model = DQNModel(observation_space, action_space, learning_rate)
        self.target_model = DQNModel(observation_space, action_space, learning_rate)
        self.target_model.set_weights(self.model.get_weights())
        self.iteration = 0

    def remember(self, state, action, reward, next_state, done):
        self.memory.push(state, action, reward, next_state, done)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_space)
        q_values = self.model.predict(np.array([state.flatten()]), verbose=0)
        return np.argmax(q_values[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = self.memory.sample(self.batch_size)
        states = np.array([x[0].flatten() for x in batch])
        actions = np.array([x[1] for x in batch])
        rewards = np.array([x[2] for x in batch])
        next_states = np.array([x[3].flatten() for x in batch])
        dones = np.array([x[4] for x in batch])

        not_dones = 1 - dones
        ns_values = self.target_model.predict(next_states, verbose=0)
        q_update = rewards + self.gamma * np.amax(ns_values, axis=1) * not_dones
        q_values = self.model.predict(states, verbose=0)

        for i, action in enumerate(actions):
            q_values[i][action] = q_update[i]

        self.model.fit(states, q_values, batch_size=self.batch_size, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        self.iteration += 1
        if self.iteration % self.target_update_iter == 0:
            self.update_target_model()

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def save_model(self, file_name):
        self.model.save_weights(file_name)


# Trading Operations

In [7]:
class TradingSimulator:
    def __init__(self, env, agent, training_episodes, validation_episodes, checkpoint_interval):
        self.env = env
        self.agent = agent
        self.training_episodes = training_episodes
        self.validation_episodes = validation_episodes
        self.checkpoint_interval = checkpoint_interval

    def train(self):
        for episode in tqdm(range(1, self.training_episodes + 1), desc="train"):
            time_step = self.env.reset()
            state = time_step.observation
            done = time_step.is_last()
            total_reward = 0

            while not done:
                action = self.agent.act(state)
                time_step = self.env.step(action)
                next_state = time_step.observation
                reward = time_step.reward
                done = time_step.is_last()

                self.agent.remember(state, action, reward, next_state, done)
                self.agent.replay()

                state = next_state
                total_reward += reward

            if episode % self.checkpoint_interval == 0:
                self.agent.save_model(f'dqn_model_checkpoint_{episode}.h5')
                print(f"Checkpoint saved at episode {episode}")

            print(f'Training Episode: {episode}, Total Reward: {total_reward}')

            if episode % self.agent.target_update_iter == 0:
                self.agent.update_target_model()

    def validate(self):
        total_rewards = []
        for episode in tqdm(range(1, self.validation_episodes + 1), desc="validate"):
            time_step = self.env.reset()
            state = time_step.observation
            done = time_step.is_last()
            total_reward = 0

            while not done:
                action = self.agent.act(state)
                time_step = self.env.step(action)
                next_state = time_step.observation
                reward = time_step.reward
                done = time_step.is_last()

                state = next_state
                total_reward += reward

            total_rewards.append(total_reward)
            print(f'Validation Episode: {episode}, Total Reward: {total_reward}')

        avg_reward = np.mean(total_rewards)
        print(f'Average Reward Over Validation Episodes: {avg_reward}')


In [8]:
env = TradingEnv(data=tickers[TARGET], data_dim=FEATURES, startingDate=START_DATE, money=CAPITAL, stateLength=STATE_LEN, transactionCosts=FEES, endingDate=END_DATE)
agent = DQNAgent(observation_space=STATE_LEN * FEATURES, action_space=ACT_SPACE)

simulator = TradingSimulator(env, agent, training_episodes=40, validation_episodes=15, checkpoint_interval=100)
simulator.train()
simulator.validate()

2024-03-08 03:25:45.680931: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-08 03:25:45.681414: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
train:   2%|▎         | 1/40 [01:42<1:06:37, 102.51s/it]

Training Episode: 1, Total Reward: -8041645.471069336


train:   5%|▌         | 2/40 [03:43<1:11:38, 113.12s/it]

Training Episode: 2, Total Reward: -4835184.062187195


train:   8%|▊         | 3/40 [05:54<1:14:48, 121.32s/it]

...

Validation Episode: 38, Total Reward: 6313.5941162109375


validate:  98%|█████████▊| 39/40 [23:35<00:36, 36.96s/it]

Validation Episode: 39, Total Reward: -70499.33087158203


validate: 100%|██████████| 40/40 [24:10<00:00, 36.25s/it]

Validation Episode: 40, Total Reward: -69790.44360351562
Average Reward Over Validation Episodes: 15197.3981924057


# Conclusion

CONCLUDE

## References

- [TensorFlow Agents](https://www.tensorflow.org/agents/overview)
- [Open Gym AI Github](https://github.com/openai/gym)
- [Greg et al, OpenAI Gym, (2016)](https://arxiv.org/abs/1606.01540)
- [Théate, Thibaut, and Damien Ernst. "An application of deep reinforcement learning to algorithmic trading." Expert Systems with Applications 173 (2021): 114632.](https://www.sciencedirect.com/science/article/pii/S0957417421000737)

## Github

Article here is also available on [Github](https://github.com/adamd1985/pairs_trading_unsupervised_learning)

Kaggle notebook available [here](https://www.kaggle.com/code/addarm/unsupervised-learning-as-signals-for-pairs-trading)

## Media

All media used (in the form of code or images) are either solely owned by me, acquired through licensing, or part of the Public Domain and granted use through Creative Commons License.

## CC Licensing and Use

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.